## Create server-side programming constructs in Azure Cosmos DB SQL API - Stored Procedures ##

#### Understand transactions in the context of JavaScript SDK ####

In a database, a transaction is typically defined as a sequence of point operations grouped together into a single unit of work. It's expected that a transaction provides **ACID** guarantees:

- **Atomicity** guarantees that all the work done inside a transaction is treated as a single unit where either all of it is committed or none.
- **Consistency** makes sure that the data is always in a healthy internal state across transactions.
- **Isolation** guarantees that no two transactions interfere with each other – generally, most commercial systems provide multiple isolation levels that can be used based on the application's needs.
- **Durability** ensures that any change that's committed in the database will always be present.

In Azure Cosmos DB SQL API, a stored procedure executes one or more operations as a single unit of work within the same scope. Stored procedures are registered in containers, and run within the scope of that specific container.

##### Stored Procedure vs SDK ACID transactions #####

Cosmos DB allows for stored procedures, triggers, and user-defined functions to run within its database engine. Interestingly, these are written using JavaScript and uploaded to the Cosmos DB collection in which they will run. Server-side programming gives a lot of extra power to a Cosmos DB-based application, including the ability to run transactions across multiple documents within the collection. In fact, server-side programming is the only way to get transaction semantics within Cosmos DB.

##### Consistency: Transactions and Indexes #####

Cosmos DB’s client-side programming model does not provide for transactional consistency across multiple documents. For example, you may have two documents to insert or update, and require that either both operations succeed or – if there is a problem with writing one of them – that both of them should fail. The Cosmos DB server-side programmability model allows for this behaviour to be implemented, because all server-side code runs within an implicit transaction. This means that we get ACID transaction semantics automatically whenever we execute a stored procedure or trigger. 


##### How is that TransactionalBatch executed? (await container.CreateTransactionalBatch(new PartitionKey(partitionKey))) #####
When ExecuteAsync is called, all operations in the TransactionalBatch are grouped, serialized into a single payload, and sent as a single request to the Azure Cosmos DB service.
The service receives the request and executes all operations within a transactional scope, and returns a response using the same serialization protocol. This response is either a success, or a failure, and contains all the individual operation responses internally.



For the stored procedure, the operations must be complete with the **server request timeout** duration.
Transactions are defined as JavaScript functions. The function is then executed when the stored procedure is invoked.

>`function name() {`
>
>`}`

Within the function, the `getContext()` method retrieves a context object, which can be used to perform multiple actions, including:

- Access the HTTP response object

- Access the corresponding Azure Cosmos DB SQL API container

Using the context object, you can invoke the `getResponse()` method to access the HTTP response object to perform actions such as returning a HTTP OK (200) and setting the response's body to a static string.

>`function greet() {`

>`    var context = getContext();`

>`    var response = context.getResponse();`

>`    response.setBody("Hello, Learn!");`

>`}`

We can also create new items using stored procedures.

Again, use the context object, you can invoke the `getCollection()` method to access the container using the JavaScript query API.

>` function createProduct(item) {`

>`     var context = getContext();` 

>`     var container = context.getCollection(); `

>`     container.createDocument(`

>`         container.getSelfLink(),`

>`         item`

>`    );`

>` }`

Although the stored procedure above will run fine, it will not let us know if it did not finish within the server response time.

>` function createProduct(item) {`

>`    var context = getContext();`

>`    var container = context.getCollection(); `

>`    var accepted = container.createDocument(`

>`        container.getSelfLink(),`

>`        item,`

>`        (error, newItem) => {`

>`            if (error) throw error;`

>`            context.getResponse().setBody(newItem)`

>`        }`

>`    );`

>`    if (!accepted) return;`

>`}`

#### Demo - Connect to CosmosDB, feed it and create a stored procedure ####

In order to create a stored procedure, we need to have a CosmosDB account provisioned and a database with a container existing.
In this example, we are using a database named **cosmicworks** with a container ***products***.

We will be first connecting to that database, and feed it using the ***cosmicworks*** utility. Then we will create the stored procedure that inserts an item inside the database. We will do this portion from the Azure Portal and verify the result by running a SQL query.

Now let us connect to my existing Cosmos DB account and upload some data to it : 

In [ ]:
$resourceGroupName = "DP420LabArchiveCosmos"    # Resource Group must already exist
$accountName = "cosmosdbaccountanna001"         # Must be all lower case
$keyKind = "primary"                            # Other key kinds: secondary, primaryReadOnly, secondaryReadOnly

#The two below lines will print out my keys, so let's not do that for security reasons - but keeping it for info purposes
#Write-Host "List keys" 
#Get-AzCosmosDBAccountKey -ResourceGroupName $resourceGroupName -Name $accountName -Type "Keys"

#The below variables are going to retrieve and store our connection string, primary key and endpoint of the Cosmos DB account.
#We will use them later on in the further C# cells to connect to the Cosmos DB account using the .NET SDK (i.e. C#)

$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName $resourceGroupName -Name $accountName -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName $resourceGroupName -Name $accountName -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName $resourceGroupName -Name $accountName).DocumentEndpoint

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos , 3.22.1"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

Installed Packages Microsoft.Azure.Cosmos, 3.22.1 Newtonsoft.Json, 13.0.1

In [ ]:
dotnet tool install --global cosmicworks

Tool 'cosmicworks' is already installed.


In [ ]:
cosmicworks --endpoint $documentEndpoint --key $primaryMasterKey --datasets product

Endpoint:	https://cosmosdbaccountanna001.documents.azure.com:443/
Auth Key:	T2NHOPlvTXHIBxAKD63a9m3HmTG9kWmiDU2X0ZJOlUXTPDSzW9QJu5b2s4YNTU94uk5W5IRXhoJb2Ues1vD9rw==

Revision:	v4
Datasets:
	product

Database:	[cosmicworks]	Status:	Created
Container:	[products]	Status:	Ready

Entity:	[9E6692D7-57E1-4D35-ACD8-105D44A1073B]	Container:products	Status:	RanToCompletion
Entity:	[A042C88C-B060-4A64-B314-ED92124047E5]	Container:products	Status:	RanToCompletion
Entity:	[91AA100C-D092-4190-92A7-7C02410F04EA]	Container:products	Status:	RanToCompletion
Entity:	[2C981511-AC73-4A65-9DA3-A0577E386394]	Container:products	Status:	RanToCompletion
Entity:	[14174164-F6C0-47FC-83FB-604C6A63408D]	Container:products	Status:	RanToCompletion
Entity:	[98324A24-9D56-4662-93A5-9A7370E7EE5A]	Container:products	Status:	RanToCompletion
Entity:	[ACC683CB-6199-416E-AE64-7C10D0C72CF9]	Container:products	Status:	RanToCompletion
Entity:	[1A176FDB-D9A8-4888-BDD9-CE4F12E97AAE]	Container:products	Status:	RanToCompletion
Enti

At this stage we created a stored procedure and ran it from the portal. If you have not done so, please do before checking the results.

If we want to check the item that was just added using the stored procedure above, inside our Cosmos DB **cosmicworks** in the container ***products***, we can use the following SQL script from the portal : 

> `SELECT * FROM products p where p.categoryId like 'demo'`

#### Creating Stored Procedures with the Javascript SDK ####

Creating a stored procedure using the .NET SDK requires the use of a special Scripts property in the **Microsoft.Azure.Cosmos.Container** class.

Let’s start with an example that assumes a container instance in a variable named container.

First, we need to store the stored procedure in a *string* variable : 

>`string sproc = @"function greet() {`

>`    var context = getContext();`

>`    var response = context.getResponse();`

>`    response.setBody('Hello, Learn!');`

>`}";`

In [ ]:
string sproc = @"function greet() {
    var context = getContext();
    var response = context.getResponse();
    response.setBody('Hello, Learn!');
}";

Next, create an object of type **Microsoft.Azure.Cosmos.Scripts.StoredProcedureProperties** with the **Id** and **Body** properties set to the unique identifier and content of the stored procedure, respectively.

>`StoredProcedureProperties properties = new()`

>`{`

>`    Id = "demoGreetingSDK",`

>`    Body = sproc`

>`};`


In [ ]:
using System;
using System.Linq;
using Microsoft.Azure.Cosmos;
using Microsoft.Azure.Cosmos.Scripts;

StoredProcedureProperties properties = new()
{
    Id = "demoGreetingSDK1",
    Body = sproc
};

/*An alternative way to define the Stored Procedure Properties is below*/
//StoredProcedureProperties properties = new("greet", sproc);

string myDBName = "cosmicworks";

CosmosClient client = new (connectionString);
AccountProperties account = await client.ReadAccountAsync();


Database database = client.GetDatabase(myDBName);
Container container = database.GetContainer("products");

await container.Scripts.CreateStoredProcedureAsync(properties);

If you'd like to parse the results, the CreateStoredProcedureAsync<> method returns an object of type **Microsoft.Azure.Cosmos.Scripts.StoredProcedureResponse** that contains metadata about the newly created stored procedure within the container.

## Create server-side programming constructs in Azure Cosmos DB SQL API - User Defined Functions ##

Azure Cosmos DB provides language-integrated, transactional execution of JavaScript. 

When using the SQL API in Azure Cosmos DB, you can write triggers and user-defined functions (UDFs) in the JavaScript language. In this module, you will author JavaScript logic that enhances the functionalities of the SQL query languages and point operations.

User-defined functions (UDFs) are used to extend the Azure Cosmos DB SQL API’s query language grammar and implement custom business logic. UDFs can only be called from inside queries as they enhance and extend the SQL query language.

Here is an example JSON document for a product with a name and a price property.

>`[`

>`  {`

>`    "name": "Black Bib Shorts (Small)",`

>`    "price": 80.00,`

>`    "priceWithTax": 92.00`

>`  }`

>`]`

A user-defined function is defined as a JavaScript function that takes in one or more scalar input[s] and then returns a scalar value as the output.

>` function name(input) {`

>`    return output;`

>`}`

In this example function, the scalar input is assumed to be a number that is then multipled by 1.15 to add 15% tax.

>`function addTax(preTax) {`

>`    return preTax * 1.15;`

>`}`

The updated query includes a third projected field that references the udf function by using the **udf.addTax()** syntax passing in the **p.price** field as an input parameter and aliasing the output of that field to the name **priceWithTax**.

>`SELECT `

>`    p.name,`

>`    p.price,`

>`    udf.addTax(p.price) AS priceWithTax`

>`FROM`

>`    products p`



#### Implement and then use a UDF using the SDK ####

The **Scripts** property in the **Microsoft.Azure.Cosmos.Container** class contains a **CreateUserDefinedFunctionAsync** method that is used to create a new user-defined function from code.

To start, we will define the JavaScript function for the UDF in a string variable.

>`string udf = @"function addTax(preTax) {`

>`    return preTax * 1.15;`

>`}";`

In [ ]:
string udfSDK = @"function addTaxSDK(preTax) {
    return preTax * 1.15;
}";

Now let's create an object of type **Microsoft.Azure.Cosmos.Scripts.UserDefinedFunctionProperties** with the **Id** and **Body** properties set to the unique identifier and content of the UDF, respectively.

In [ ]:
UserDefinedFunctionProperties properties = new()
{
    Id = "addTaxSDK",
    Body = udfSDK
};

Finally, invoke the **CreateUserDefinedFunctionAsync** method of the container variable to create a new UDF passing in the properties composed earlier.

In [ ]:
await container.Scripts.CreateUserDefinedFunctionAsync(properties);

Now, below is a complete code that you would use in the C# project and it uses a slightly different way of doing the same thing we did earlier.

In [ ]:
using System;
using Microsoft.Azure.Cosmos;
using Microsoft.Azure.Cosmos.Scripts;

string myDBName = "cosmicworks";

CosmosClient client = new (connectionString);
AccountProperties account = await client.ReadAccountAsync();


Database database = client.GetDatabase(myDBName);
Container container = database.GetContainer("products");

/*We can also create first a variable props of type UserDefinedFunctionProperties using an empty constructor*/

UserDefinedFunctionProperties props = new ();

/* And after that, we can set the properties of that varable, that is the ID and the BODY of the UDF*/

props.Id = "taxSDKAlternativeWay";
props.Body = "function taxSDKAlternativeWay(i) { return i * 1.25; }";

/* Asynchronously call the container variable’s Scripts.CreateUserDefinedFunctionAsync method by passing in the 
props variable as a parameter and saving the result in a variable named udf of type UserDefinedFunctionResponse*/

UserDefinedFunctionResponse udf = await container.Scripts.CreateUserDefinedFunctionAsync(props);
   
Console.WriteLine($"Created UDF [{udf.Resource?.Id}]");

Created UDF [taxSDKAlternativeWay]


## Add triggers to an operation ##

Triggers are the core way that Azure Cosmos DB SQL API can inject business logic both before and after operations. Triggers are resources stored within a container, and their code is written in JavaScript, much like stored procedures and user-defined functions.

Triggers are defined as JavaScript functions. The function is then executed when the trigger is invoked.

>`function name() {`

>`}`

Within the function, the `getContext()` method retrieves a context object, which can be used to perform multiple actions, including:

- Access the HTTP request object (the source of a pre-trigger)

- Access the HTTP response object (the source of a post-trigger)

- Access the corresponding Azure Cosmos DB SQL API container

Using the context object, you can invoke the getRequest() or getResponse() methods to access the HTTP request and response objects. You can also invoke the getCollection() method to access the container using the JavaScript query API.

Using the context object, you can invoke the `getRequest()` or `getResponse()` methods to access the HTTP request and response objects. You can also invoke the `getCollection()` method to access the container using the JavaScript query API.

#### Pre-trigger ####

Pre-triggers are ran before an operation and cannot have any input parameters. They can perform actions such as validate the properties of an item, or inject missing properties.

Let's walk through a simple example where a JSON item is ready to be created in a container.

>`{`

>`  "id": "caab0e5e-c037-48a4-a760-140497d19452",`

>`  "name": "Handlebar",`

>`  "categoryId": "e89a34d2-47ee-4da8-bcf6-10f552604b79",`

>`  "categoryName": "Accessories",`

>`  "price": 50`

>`}`

In this example, a pre-trigger will be created that runs before an HTTP POST operation. This trigger will check for the existence of a **label** property. If it does not exist, it will add the label property with a value of new. The JavaScript code for this function uses the `getContext()` and `getRequest()` methods to get the current HTTP request, and then the request body.

>`function addLabel(item) {`

>`    var context = getContext();`

>`    var request = context.getRequest();`

    
>`    var pendingItem = request.getBody();`

>`}`

Finally, the function will check for the existence of the label property, add it if it does not exist, and then return the modified item as the updated request body.

>`if (!('label' in pendingItem))`

>`    pendingItem['label'] = 'new';`

>`request.setBody(pendingItem);`

If you invoke the create operation using this pre-trigger, you should expect your resulting JSON to include the label property thanks to the logic in the trigger.

#### Post-trigger ####

Post-triggers run after an operation has completed and can have input parameters even though they are not required. They have action to the HTTP response message right before it is sent to the client. They can perform actions such as updating or creating secondary items based on changes to your original item.

Let's walk through a slightly different example with the same JSON file. Now, a post-trigger will be used to create a second item with a different materialized view of our data. Our goal, is to create a second item with three JSON properties; sourceId, categoryId, and displayName.

>`{`

>`  "sourceId": "caab0e5e-c037-48a4-a760-140497d19452",`

>`  "categoryId": "e89a34d2-47ee-4da8-bcf6-10f552604b79",`

>`  "displayName": "Handlebar [Accessories]",`

>`}`

We can start our function by getting both the container and HTTP response using the `getCollection()` and `getResponse()` methods. We will also get the newly created item using the `getBody()` method of the HTTP response object.

>`function createView() {`

>`    var context = getContext();`

>`    var container = context.getCollection();`

>`    var response = context.getResponse();`

>`    var createdItem = response.getBody();`

>`    var viewItem = {`

>`        sourceId: createdItem.id,`

>`        categoryId: createdItem.categoryId,`

>```        displayName: `${createdItem.name} [${createdItem.categoryName}]` ```

>`    };`
 
>`    var accepted = container.createDocument(`

>`        container.getSelfLink(),`

>`        viewItem,`

>`        (error, newItem) => {`

>`            if (error) throw error;`

>`        }`

>`    );`

>`    if (!accepted) return;`

>`}`

##### Create a pre-trigger using the SDK ######

In [ ]:
string preTrigger = @"function addLabelSDK1() {
    var context = getContext();
    var request = context.getRequest();
    
    var pendingItem = request.getBody();

    if (!('label' in pendingItem))
        pendingItem['label'] = 'new';

    request.setBody(pendingItem);
}";

Invoke the **CreateTriggerAsync** method of the container variable to create a new pre-trigger passing in the properties composed earlier.

In [ ]:
TriggerProperties properties = new()
{
    Id = "addLabelSDK1",
    Body = preTrigger,
    TriggerOperation = TriggerOperation.Create,
    TriggerType = TriggerType.Pre
};

await container.Scripts.CreateTriggerAsync(properties);